In [1]:
items = open("C:\\Users\\Gauraw\\Desktop\\test.txt", 'r')
readitems=(items.readlines())
print(readitems)

['The launch of international trade in Indian rupees could lead to annual savings of $30-36 billion in hard currency and widen the scope for such trades with countries in the region, easing pressure on the exchange rate, experts told ET.\n', '\n', "The mechanism, which can be kicked off without delay with Russia, may be favourable for Indian government bonds as well. The surplus balance held in a special vostro account to be opened under the rupee payment mechanism can be used for investing in the local capital market by entities based in India's trading partners under the bespoke model.\n", '\n', '"When a country faces a record high current account deficit, such rupee-denominated trades with select countries save dollar outflows on account of imports," said Bank of Baroda NSE -1.09 % chief economist Madan Sabnavis.']


In [2]:
import re
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gauraw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
!pip install snorkel

In [4]:
string = []    
with open("C:\\Users\\Gauraw\\Desktop\\test.txt", "r") as f:
    full_text = f.read()
    for l in re.split(r"(\.)", full_text):
        if l != ".":
            string.append(l + "\n")
pd.DataFrame(string)

,0
0,The launch of international trade in Indian ru...
1,"\n\nThe mechanism, which can be kicked off wit..."
2,The surplus balance held in a special vostro ...
3,"\n\n""When a country faces a record high curren..."
4,09 % chief economist Madan Sabnavis\n
5,\n


In [5]:
df=pd.DataFrame(string, columns=["article"])
print(df)

                                             article
0  The launch of international trade in Indian ru...
1  \n\nThe mechanism, which can be kicked off wit...
2   The surplus balance held in a special vostro ...
3  \n\n"When a country faces a record high curren...
4              09 % chief economist Madan Sabnavis\n
5                                                 \n


In [6]:
df.head()

,article
0,The launch of international trade in Indian ru...
1,"\n\nThe mechanism, which can be kicked off wit..."
2,The surplus balance held in a special vostro ...
3,"\n\n""When a country faces a record high curren..."
4,09 % chief economist Madan Sabnavis\n


In [7]:
stop_words = stopwords.words('english')
df['article'] = df['article'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [8]:
df.replace('\d+', '', regex=True, inplace=True)

In [9]:
df['article'] = df['article'].apply(str.lower)
print(df)

                                             article
0  the launch international trade indian rupees c...
1  the mechanism, kicked without delay russia, ma...
2  the surplus balance held special vostro accoun...
3  "when country faces record high current accoun...
4                   % chief economist madan sabnavis
5                                                   


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
df_train,df_test = train_test_split(df,train_size=0.5)

In [12]:
print(df_train.shape)

(3, 1)


In [13]:
from snorkel.labeling import labeling_function

In [14]:
trade = 1
ABSTAIN = -1

In [15]:
@labeling_function()
def lf_keyword_lookup(x):
  keywords = ["currency","international"]
  return trade if any(word in x.article.lower() for word in keywords) else ABSTAIN

In [16]:
@labeling_function()
def lf_keyword2_lookup(x):
  keywords = ["dollar","rupee"]
  return trade if any(word in x.article.lower() for word in keywords) else ABSTAIN

In [17]:
@labeling_function()
def lf_keyword3_lookup(x):
  keywords = ["country","account"]
  return trade if any(word in x.article.lower() for word in keywords) else ABSTAIN

In [18]:
lfs = [lf_keyword_lookup,lf_keyword2_lookup,lf_keyword3_lookup]

In [19]:
from snorkel.labeling import PandasLFApplier

In [20]:
applier = PandasLFApplier(lfs=lfs)

In [21]:
L_train = applier.apply(df=df_train)

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 2993.08it/s]


In [22]:
from snorkel.labeling.model import LabelModel

In [23]:
label_model = LabelModel(cardinality=2,verbose=True)
label_model.fit(L_train=L_train,n_epochs=500,log_freq=100,seed=123)

INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|                                                                                       | 0/500 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=0.388]
INFO:root:[100 epochs]: TRAIN:[loss=0.033]
INFO:root:[200 epochs]: TRAIN:[loss=0.028]
 41%|██████████████████████████████▋                                            | 205/500 [00:00<00:00, 2032.74epoch/s]INFO:root:[300 epochs]: TRAIN:[loss=0.026]
INFO:root:[400 epochs]: TRAIN:[loss=0.025]
100%|███████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 2053.36epoch/s]
INFO:root:Finished Training


In [24]:
df_train['Labels'] = label_model.predict(L=L_train,tie_break_policy="abstain")

C:\Users\Gauraw\AppData\Local\Temp/ipykernel_6928/1145111523.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Labels'] = label_model.predict(L=L_train,tie_break_policy="abstain")


In [25]:
df_train

,article,Labels
0,the launch international trade indian rupees c...,1
3,"""when country faces record high current accoun...",1
5,,-1
